In [18]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "accuser"
password = "snhupass"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.A(html.Img(width='150', src='data:image/png;base64,{}'.format(encoded_image.decode())), href='https://snhu.edu')),
    html.Center(html.B(html.H1('Chuluda Rescue Animal Dashboard'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
        id='radio-items',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        #column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        hidden_columns=["datetime", "location_lat", "location_long", "age_upon_outcome", "monthyear"],
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 dcc.Graph(
                     id="graph-id",
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
        ])
    
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('datatable-id', 'data'),
    [Input(component_id='radio-items', component_property='value')]
)
def radioDisplay(option):

    if option == 'water':
        df = pd.DataFrame(list(shelter.read({ 
        "sex_upon_outcome": "Intact Female",
        "$or": [{"breed": "Labrador Retriever Mix"}, 
                {"breed": "Labrador Retriever Mix"}, 
                {"breed": "Newfoundland"}],
        "$and": [{"age_upon_outcome_in_weeks": {"$gte": 26}},
                 {"age_upon_outcome_in_weeks": {"$lte": 156}}]
        })))
        
    elif option == 'mountain':
        df = pd.DataFrame(list(shelter.read({ 
        "sex_upon_outcome": "Intact Male",
        "$or": [{"breed": "German Shepherd"}, 
                {"breed": "Alaskan Malamute"}, 
                {"breed": "Old English Sheepdog"},
                {"breed": "Siberian Husky"},
                {"breed": "Rottweiler"}],
        "$and": [{"age_upon_outcome_in_weeks": {"$gte": 26}},
                 {"age_upon_outcome_in_weeks": {"$lte": 156}}]
        })))
        
    elif option == 'disaster':
        df = pd.DataFrame(list(shelter.read({ 
        "sex_upon_outcome": "Intact Male",
        # Doberman Pinschers are incorrectly listed as "Doberman Pinsch" in database
        "$or": [{"breed": "Doberman Pinsch"}, 
                {"breed": "German Shepherd"}, 
                {"breed": "Golden Retriever"},
                {"breed": "Bloodhound"},
                {"breed": "Rottweiler"}],
        "$and": [{"age_upon_outcome_in_weeks": {"$gte": 20}},
                 {"age_upon_outcome_in_weeks": {"$lte": 300}}]
        })))
        
    # THIS ELIF STATEMENT IS NOT NECESSARY
    # IF NOT WATER, MOUNTAIN OR DISASTER, WILL ALREADY DEFAULT TO ELSE
    #elif option == 'reset':
        #df = pd.DataFrame.from_records(shelter.read({}))
    
    else:
        df = pd.DataFrame.from_records(shelter.read({}))

    return df.to_dict('records')


#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output("graph-id", "figure"),
    [Input("datatable-id", "derived_viewport_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    chart = px.pie(dff, names='breed')
    return chart


@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data'),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    
    #If no row selected, use top row
    if selected_rows == []:
        selected_rows = [0]
        
    locLat = dff.iloc[selected_rows[0],13]
    locLon = dff.iloc[selected_rows[0],14]
    name = dff.iloc[selected_rows[0],9]
    breed = dff.iloc[selected_rows[0],4]
    sex = dff.iloc[selected_rows[0],12]
      
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with popup
            dl.Marker(position=[locLat, locLon], children=[
                dl.Popup([
                    html.H2(breed),
                    html.H3("Name: " + name),
                    html.H3("Sex: " + sex)
                ])
            ])
        ])
    ]

app
